# Filtro de spam


#### [Entrenamiento:](#Entrenamiento)
- def que separe una línea en palabras y las agregue a un conjunto dado por parámetro
- def de un generator que devuelva un set de palabras por cada correo de un mbox dado por parámetro
- def que devuelve Dict de la forma {'w' : n } con w = palabra y n = Hw para 'ham.mbox' o n = Sw para 'spam.box'

#### [Clasificación:](#Clasificación)
- def que devuelva P(y|xw) para cada palabra w de un conjunto de palabras, uno por cada correo de 'news.box'
- def que devuelva lista de 15 palabras de mejor clasificación individual de un correo nuevo
- def que devuelva P(y|x1, . . . , x15) de la lista anterior
- def que devuelva lista de 0 o 1 por cada correo de 'news.mbox' en el mismo orden

#### [Incorporación:](#Incorporación)
- def que devuelva un set de palabras de un 'new-spam.mbox' o 'new-ham.mbox' que contiene un solo mensaje
- def que combine el anterior conjunto al dict de entrenamiento aumentando en 1 cada valor
- def que escriba el nuevo correo 'new-spam.mbox' o 'new-ham.mbox' en el fichero que corresponda

#### [Ejemplo de uso:](#Ejemplo-de-uso-de-los-3-procedimientos)
- Ejemplo de uso de los 3 procedimientos anteriores

# Entrenamiento

### Utilidad

In [ ]:
import re
import mailbox


# Recibe un string con el subject y body del un solo correo
# Devuelve un set con las palabras de ese string (separadas según un regex)
def separate_in_words(payload, regex):
    words = set()
    for w in re.split(regex, payload):
        words.add(w)
    words.remove('')
    return words


# Recibe un string con la ruta de un .mbox
# Devuelve generator de un set de palabras por correo recorrido
def words_per_mail(mbox, regex):    
    for mail in mailbox.mbox(mbox):
        payload = mail['subject'] + ' ' + mail.get_payload()
        words = separate_in_words(payload, regex)
        yield words


# Recibe un set de palabras y un dict a modificar
# Agrega words a dictionary, incrementando la ocurrencia en 1
def merge_to_dict(words, dictionary):
    for w in words:
        dictionary[w] = dictionary[w] + 1 if w in dictionary else 1


# Recibe un string con la ruta de un .mbox
# Devuelve dict con key = <una palabra> y value = <nº de correos en los que aparece>
def training_dict(mbox, regex):
    dictionary = {}
    for words in words_per_mail(mbox, regex):
        merge_to_dict(words, dictionary)
    return dictionary



### Procedimiento

In [ ]:
# Devuelve dict de la forma {'word' : Sw }
# y otro dict de la forma {'word' : Hw }
def training(spambox, hambox, regex = '\W+'):
    spam_dict = training_dict(spambox, regex)
    ham_dict = training_dict(hambox, regex)
    return spam_dict, ham_dict



# Clasificación

### Utilidad

In [ ]:
# Devuelve list de N palabras con mejor clasificación
def top_N(words, spam_dict, ham_dict, num):
    top = []  # De la forma: [ (Pw1, w1), (Pw2, w2), (Pw3, w3), ...]
    for w in words:
        sw = spam_dict[w] if w in spam_dict else 0
        hw = ham_dict[w]  if w in ham_dict  else 0
        p  = 0 if sw == 0 and hw == 0 else abs((sw-hw)/(2*sw+2*hw))  # p∈[0, 0.5] con 0.5 ≡ mejor clasificación
        top.append(( p, w ))    
    # Devuelve máximo N valores
    return [word for prob, word in sorted(top,reverse=True)][:num]


# Naïve-Bayes de P(y|x1,...,xn)
def naive_bayes(words):
    sw_acum = 1
    hw_acum = 1
    for w in words:
        sw = spam_dict[w] if w in spam_dict else 1
        hw = ham_dict[w]  if w in ham_dict  else 1
        sw_acum = sw_acum * sw
        hw_acum = hw_acum * hw
    return sw_acum/(sw_acum + hw_acum)


### Procedimiento

In [ ]:
# Función que recibe un mbox de correos nuevos
# y que devuelva una lista de 0 o 1 para cada correo nuevo 
def clasification(newsbox, spam_dict, ham_dict, regex = '\W+', num = 15):
    clas = []    
    for words in words_per_mail(newsbox, regex):
        top_N_words = top_N(words, spam_dict, ham_dict, num)
        probability  = naive_bayes(top_N_words)
        is_spam      = 1 if probability > 0.9 else 0
        clas.append(is_spam)        
    return clas


# Incorporación

### Utilidad

In [ ]:
# Escribe el nuevo correo ubicado en la ruta 'new'
# en el fichero de ruta 'box'
def append_new_to_mbox(new_mbox, mbox):
    with open(mbox, 'a') as writable:
        with open(new_mbox) as readable:
            writable.write(readable.read())


# Actualiza dictionary a partir del set de palabras
# del único mensaje contenido en 'new_box'
def update_dict(new_mbox, dictionary, regex):
    for words in words_per_mail(new_mbox, regex):
        merge_to_dict(words, dictionary)
        


### Procedimiento

In [ ]:
# Actualiza dictionary y escribe el contenido de new_box en mbox
def incorporation(new_mbox, mbox, dic = None, regex = '\W+'):
    append_new_to_mbox(new_mbox, mbox)
    if dic is not None:
        update_dict(new_mbox, dic, regex)


# Ejemplo de uso de los 3 procedimientos

### Variables

- Rutas de archivos

In [ ]:
hambox = './ham.mbox'
spambox = './spam.mbox'
newsbox = './news.mbox'
new_hambox  = './new-ham.mbox'
new_spambox = './new-spam.mbox'

- Parametros opcionales de training(), clasification() e incorporation()

In [ ]:
# Criterio de separación alternativo
regex = '[.@_#/]?[\s]+|[,;:()<>*~][\s]*'

# Número de palabras alternativo para hacer Näive-Bayes
num = 19

### Uso de training()
- Parámetro regex es opcional, por defecto es '\W+', cualquier secuencia de caracteres no alfanúmericos.

In [ ]:
# Entrenamiento
spam_dict, ham_dict = training(spambox, hambox, regex = regex)

print(len(spam_dict), len(ham_dict))

### Uso de clasification()
- Parámetro regex es opcional, por defecto es '\W+', cualquier secuencia de caracteres no alfanúmericos.
- Parámetro num es opcional, por defecto es 15.
- Requiere que training() haya sido ejecutado antes.

In [ ]:
# Clasificación
res = clasification(newsbox, spam_dict, ham_dict, regex = regex, num = num)

print(res)

### Uso de incorporation()
- Parámetro regex es opcional, por defecto es '\W+', cualquier secuencia de caracteres no alfanúmericos.
- La ejecución previa de training() es opcional.
- Al usar incorporation() se actualizan, con las palabras nuevas, los dicts devueltos por training(), si existen.

<p>NOTA: Modifica el contenido de los archivos spambox y hambox</p>

In [ ]:
# Incorporación a spam.box
incorporation(new_spambox, spambox, dic = spam_dict, regex = regex)

# Incorporación a ham.box
incorporation(new_hambox, hambox, dic = ham_dict, regex = regex)

print(len(spam_dict), len(ham_dict))